# Submission Notebook

In [1]:
%cd /kaggle
!cp -r /kaggle/input/nfldata-dcnv2/DCNv2 /kaggle/.
%cd /kaggle/DCNv2
! chmod a+x *
! ./make.sh &> /dev/null

/kaggle
/kaggle/DCNv2


In [2]:
# Install motmetrics
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/py-cpuinfo-8.0.0.tar.gz
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/xmltodict-0.12.0-py2.py3-none-any.whl
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/pytest_benchmark-3.4.1-py2.py3-none-any.whl
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/flake8_import_order-0.18.1-py2.py3-none-any.whl
! pip install -qq /kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install/motmetrics-1.2.0-py3-none-any.whl

In [3]:
%cd /kaggle
!cp -r /kaggle/input/nfldata-fairmot-package/FairMOT /kaggle/.
%cd /kaggle/FairMOT
! pip install -qq --no-deps --no-index --find-links=/kaggle/input/nfl07b-fairmot-dependencies/pre_fairmot_install -r requirements.txt #&> /dev/null

/kaggle
/kaggle/FairMOT


In [4]:
! pip uninstall -y datasets

Found existing installation: datasets 1.12.1
Uninstalling datasets-1.12.1:
  Successfully uninstalled datasets-1.12.1


In [5]:
# User
! cp -r /kaggle/input/nfl-helmet-assignment-kaggle/nfl_helmet_assignment_kaggle /kaggle/.

In [6]:
import os, sys
import subprocess
from multiprocessing import Pool
import random
import numpy as np
import pandas as pd
import torch
from IPython.display import display, Video
from tqdm.auto import tqdm

# FairMOT imports
sys.path.append('/kaggle/DCNv2')
sys.path.append('/kaggle/FairMOT/src/lib')
from datasets.dataset.jde import LoadVideo

# nfl_helmet_assignment_kaggle imports
sys.path.append('/kaggle/input/easydict-master/easydict-master')
sys.path.append(
    ('/kaggle/input/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master'
     '/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch')
)
sys.path.append('/kaggle/nfl_helmet_assignment_kaggle/')
from helmet_tracker.utils.score import check_submission
from helmet_tracker.utils.video import ffmpeg_friendly_encode
from helmet_tracker.utils.features import add_track_features
from helmet_tracker.models.helmet_mapping import mapping_df_2d
from helmet_tracker.models.fairmot_postprocess import fmot_postprocess_hmap

In [7]:
# For traindebug videos

# BASE_DIR = '/kaggle/input/nfl-health-and-safety-helmet-assignment'

# dir_video = os.path.join(BASE_DIR, 'train')
# tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
# helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')

# # For developing, restrict to "traindebug" samples

# pth_ss_hmap = '/kaggle/input/nfl-mydata/submission_helmtrack_debug.csv'
# ss_hmap = pd.read_csv(pth_ss_hmap)
# videos = ss_hmap['video_frame'].str.split('_').str[:3].str.join('_').unique()[:2]

# helmets['video'] = helmets['video_frame'].str.split('_').str[:3].str.join('_')
# helmets = helmets[helmets['video'].isin(videos)]

In [8]:
BASE_DIR = '/kaggle/input/nfl-health-and-safety-helmet-assignment'

dir_video = os.path.join(BASE_DIR, 'test')
tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

videos = [pth.split('.')[0] for pth in os.listdir(dir_video)]

if len(videos) == 6:
    videos = videos[:1]

helmets['video'] = helmets['video_frame'].str.split('_').str[:3].str.join('_')
helmets = helmets[helmets['video'].isin(videos)]

In [9]:
videos

['57906_000718_Sideline']

# Helmet mapping 2D

In [10]:

# Add estimated video frame to NGS data

tracking = add_track_features(tracking) 

CONF_THRE = 0.3

df_list = list(helmets.groupby('video_frame'))
df_list = [(video_frame, df, tracking, CONF_THRE) for video_frame, df in df_list]

def _mapping_df(args):
    video_frame, df, tracking, conf_thre = args
    return mapping_df_2d(video_frame, df, tracking, conf_thre=conf_thre)

p = Pool(processes=os.cpu_count())
ss_hmap = []
with tqdm(total=len(df_list)) as pbar:
    for this_df in p.imap(_mapping_df, df_list):
        ss_hmap.append(this_df)
        pbar.update(1)
p.close()

ss_hmap = pd.concat(ss_hmap)


  0%|          | 0/440 [00:00<?, ?it/s]

# FairMOT

In [11]:
! mkdir -p /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/models-fairmot/dla34-ba72cf86.pth /root/.cache/torch/hub/checkpoints/.

load_model = '/kaggle/input/nfldata-fairmot/exp/mot/nfl_ft_mix_dla34/model_last.pth'
conf_thres = 0.3

dir_demo = '/kaggle/FairMOT/demo'

for video in tqdm(videos, total=len(videos)):
    
    input_video = os.path.join(dir_video, f'{video}.mp4')
    output_root = os.path.join(dir_demo, video)

    ! mkdir -p {output_root}
    %cd /kaggle/FairMOT/src
    ! python demo.py mot --load_model {load_model} --conf_thres {conf_thres} --input-video {input_video} --output-root {output_root}


    

  0%|          | 0/1 [00:00<?, ?it/s]

/kaggle/FairMOT/src
Fix size testing.
training chunk_sizes: [6, 6]
The output will be saved to  /kaggle/FairMOT/src/lib/../../exp/mot/default
heads {'hm': 1, 'wh': 4, 'id': 128, 'reg': 2}
2021-11-02 15:16:43 [INFO]: Starting tracking...
Lenth of the video: 440 frames
Creating model...
loaded /kaggle/input/nfldata-fairmot/exp/mot/nfl_ft_mix_dla34/model_last.pth, epoch 16
2021-11-02 15:16:52 [INFO]: Processing frame 0 (100000.00 fps)
2021-11-02 15:16:56 [INFO]: Processing frame 20 (11.98 fps)
2021-11-02 15:17:00 [INFO]: Processing frame 40 (12.48 fps)
2021-11-02 15:17:05 [INFO]: Processing frame 60 (12.00 fps)
2021-11-02 15:17:09 [INFO]: Processing frame 80 (12.10 fps)
2021-11-02 15:17:14 [INFO]: Processing frame 100 (12.11 fps)
2021-11-02 15:17:18 [INFO]: Processing frame 120 (11.95 fps)
2021-11-02 15:17:22 [INFO]: Processing frame 140 (11.90 fps)
2021-11-02 15:17:27 [INFO]: Processing frame 160 (11.70 fps)
2021-11-02 15:17:31 [INFO]: Processing frame 180 (11.64 fps)
2021-11-02 15:17:36

# Post-process helmet mapping with FairMOT

In [12]:
video_frame = ss_hmap['video_frame'].str.split('_')
ss_hmap['video'] = video_frame.str[:3].str.join('_')
ss_hmap['frame'] = video_frame.str[3].astype(int)

In [13]:
%%time

outs = []
for video in tqdm(videos, total=len(videos)):
    
    df_hmap = ss_hmap.query('video==@video')

    pth_fmot = os.path.join(dir_demo, video, 'results.txt')
    pth_video = os.path.join(dir_video, video + '.mp4')

    df_tracks = fmot_postprocess_hmap(df_hmap, pth_fmot, pth_video)
    
    outs.append(df_tracks)
    
ss_fmot = pd.concat(outs, axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 5.42 s, sys: 31.5 ms, total: 5.45 s
Wall time: 5.52 s


In [14]:
assert ss_fmot['label_fairmot'].isna().sum() == 0

ss_fmot.drop('label', axis=1, inplace=True)
ss_fmot.rename(columns={'label_fairmot': 'label'}, inplace=True)

columns = pd.read_csv(
    '/kaggle/input/nfl-health-and-safety-helmet-assignment/sample_submission.csv').columns.values
ss_fmot = ss_fmot[columns]

# Check submission and save to disk

In [15]:
check_submission(ss_fmot)

True

In [16]:
ss_fmot.to_csv('/kaggle/working/submission.csv', index=False)